The following Zi failed
```
儿 猴 龙 头 死 毛 皮 眉 耳 肉 一 午 四 寅 甲 癸 零 丿 彡 攵 氵犭
```

In [1]:
from time import time
import re
from pathlib import Path

import pandas as pd 

import ollama

from utils import *

In [2]:
model_name = "qwen2.5"  # "llama3.1"
pattern = r'```json\s*(.*?)\s*```'
file_log = "char360-qwen2.5.log"

In [3]:
prompt_zi = """
You are an expert in Chinese language, 

can you generate a holistic view on this chinese character  {zi}
in terms of the following attributes:
含义
字形
读音 
字源
常用词组
成语
例句
短故事
诗词
图片
音频 
视频 
电影
参考资料
有趣网站

(1) give the answer in Chinese 
(2) format the answer in valid json and ensure quotes are properly escaped (specifically avoid double-quotes nested in doube-quotes)
(3) whenever possible, give 5 or more examples for the following attributes:

常用词组
成语
例句
短故事
诗词
图片
音频 
视频 
电影
参考资料
有趣网站

"""

In [4]:
df = pd.read_csv("char360_fib_num.csv")
zi_fib_nums = df.to_dict("records")

In [5]:
for xx in zi_fib_nums:
    zi, fib_num = xx['zi'], xx['fib_num']
    fib_num_str = str(fib_num).zfill(4)
    fp = Path(f"./llm_models/{model_name}") / f"{fib_num_str}-{zi}-1.json"    
    if fp.exists():
        # log_msg(f"SKIP {zi}: Already processed - {fp} ", file_log)
        continue

    print(f"Processing {zi} ...")
    
    ts_1 = time()
    try:
        resp = ollama.generate(model=model_name, prompt=prompt_zi.format(zi=zi))
    except Exception as e:
        log_msg(f"[ERROR] {zi} : Failed to call ollama.generate()", file_log)
        continue
        
    ts_2 = time()
    del_t = ts_2 - ts_1
    del_t_str = f"{del_t:.3f}"
    json_raw = resp.get('response', "")
    if not json_raw: 
        log_msg(f"[ERROR] No LLM response: {zi}", file_log)
        continue

    log_msg(f"... Completed {zi} in {del_t_str} sec")

    match = re.search(pattern, json_raw, re.DOTALL)

    if match:
        json_string = match.group(1)
        if not json_string:
            log_msg(f"[WARN] No JSON extracted: {zi}")
            continue
            
        with open(fp, "w", encoding="utf-8") as f:
            f.write(json_string)
    else:
        log_msg(f"[ERROR] No JSON extracted: {zi}", file_log)

Processing 气 ...
SKIP 气: Already processed - llm_models/qwen2.5/0000-气-1.json 
Processing 炁 ...
SKIP 炁: Already processed - llm_models/qwen2.5/0000-炁-1.json 
Processing 空 ...
SKIP 空: Already processed - llm_models/qwen2.5/0000-空-1.json 
Processing 心 ...
SKIP 心: Already processed - llm_models/qwen2.5/0001-心-1.json 
Processing 日 ...
SKIP 日: Already processed - llm_models/qwen2.5/0002-日-1.json 
Processing 月 ...
SKIP 月: Already processed - llm_models/qwen2.5/0002-月-1.json 
Processing 阳 ...
SKIP 阳: Already processed - llm_models/qwen2.5/0002-阳-1.json 
Processing 阴 ...
SKIP 阴: Already processed - llm_models/qwen2.5/0002-阴-1.json 
Processing 人 ...
SKIP 人: Already processed - llm_models/qwen2.5/0003-人-1.json 
Processing 儿 ...
... Completed 儿 in 99.266 sec
Processing 地 ...
SKIP 地: Already processed - llm_models/qwen2.5/0003-地-1.json 
Processing 天 ...
SKIP 天: Already processed - llm_models/qwen2.5/0003-天-1.json 
Processing 女 ...
SKIP 女: Already processed - llm_models/qwen2.5/0003-女-1.json 
Proce

KeyboardInterrupt: 